# Afeka - ML3 - Titanic

Noam Levi  
205530611  
[Kaggle Profile](https://www.kaggle.com/noamlevi)

## Introduction

We're going to be working on the [Titanic dataset](https://www.kaggle.com/c/titanic/data) from the [kaggle competition](https://www.kaggle.com/c/titanic).  

This is a continued work from `Assignment1`.

---

Roadmap:
- [Data Exploration](#Data-Exploration) and Data Visualising - *from `Assignment1`*
- [Data Cleaning](#Data-Cleaning), handling missing data in our df using different methods - *from `Assignment1`*
- [Feature Engineering](#Feature-Engineering), creating/choosing the right features for a better ML model - *from `Assignment1`*
- [Training & Model Comparing](#Training-&-Model-Comparing), Fitting a linear model & loss function plotting using different hyperparameters
- [Testing](#Testing)
- [Summary](#Summary)
- [References](#References)

## Imports

In [ ]:
from IPython.display import display, Markdown
import random
import math
import pandas as pd
from pandas.api.types import is_numeric_dtype
# from pandas_profiling import ProfileReport
import sweetviz as sv
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, Normalizer, LabelEncoder #, OneHotEncoder
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_validate, LeavePOut, KFold #, train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn.feature_selection import RFE

sns.set_style("darkgrid")
# plt.style.use("fivethirtyeight")

Joining `train` & `test` to a single `data` df for an easier time while working on the model.

In [ ]:
train = pd.read_csv('../data/train.csv', index_col='Id')
test = pd.read_csv('../data/test.csv', index_col='Id')

ntrain = train.shape[0]
ntest = test.shape[0]
data = pd.concat([train, test]).reset_index(drop=True)
# data

# train = pd.get_dummies(data)[:ntrain]
# test = pd.get_dummies(data)[:ntest]
# test.index = itest

In [ ]:
# global pipeline
pipeline = Pipeline([(
    'none',
    FunctionTransformer(func=None)
)])
# use pipeline.steps.append() for adding steps

In [ ]:
def appendToPipeline(pipeline, func, func_name=None):
    if func_name == None:
        func_name = func.__name__
    
    names = []
    for (n,f) in pipeline.steps:
        names += [n]
    
    if func_name not in names:
        pipeline.steps.append((
            func_name,
            FunctionTransformer(func)
        ))

## Data Exploration

To start thing off, we can use ~~pandas_profiling~~ sweetviz library to get an overview of the entire training dataset.

In [ ]:
# data.head()

In [ ]:
# ProfileReport(train, minimal=True)

In [ ]:
# report = sv.analyze(data)
# report.show_notebook()

In [ ]:
sns.set_style("darkgrid")

display(data.head())

Let's take a look at the target value's distribution.  
It seems to be normally distributed and a bit right skewed.  

In [ ]:
# Get the fitted parameters used by the function
plt.figure(figsize=(16,8))

(mu, sigma) = ( round(item, 2) for item in stats.norm.fit( data['SalePrice'][:ntrain] ) )

display(Markdown(f'$\mu$ = {mu}'))
display(Markdown(f'$\sigma$ = {sigma}'))

# plot the distribution
sns.distplot(
    data['SalePrice'][:ntrain],
    kde_kws = {'color': '#4C4C4C'}
)
plt.legend(
    # [f'Normal dist N($\mu=${mu}, $\sigma=${sigma})'],
    [f't ~ $N(\mu=${mu}, $\sigma=${sigma}$)$'],
    fontsize = 'x-large'
)
# plt.ylabel('Frequency')
plt.title('SalePrice Distribution')

Next up is a correlation heatmap.  

Since we have so many different feature already in our dataset, we should choose only the best features.  
Using the following heatmap we can choose the feature that have the best correlation to the target feature.

In [ ]:
corr = data.corr()

plt.figure(figsize=(30, 12))
sns.heatmap(
    corr,
    cmap = 'coolwarm',
    annot = True,
)
plt.show()

Some more graph that depict the correlation of several features to the target label.

In [ ]:
plt.figure(figsize=(16,8))
sns.boxplot(
    x = 'GarageCars',
    y = 'SalePrice',
    data = data
)
plt.title('GarageCars / SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(
    x = 'FullBath',
    y = 'SalePrice',
    data = data
)
plt.title('FullBath / SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
# sns.boxplot(
#     x = 'OverallQual',
#     y = 'SalePrice',
#     data = train
# )
# plt.title('OverallQual / SalePrice')
# plt.show()

sns.regplot(
    x = 'OverallQual',
    y = 'SalePrice',
    data = data,
    line_kws = {
        'color': '#E37D3D' # '#B55D60'
    }
)
plt.ylim(0)

plt.title('OverallQual / SalePrice')

## Data Cleaning

Before we can do any kind of feature engineering we need to clean our dataset.  

The first to do here is to have a general idea of the total and percentage of missing data in each column.


In [ ]:
# def NAs(data):
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing %': data_na})
missing_data.head()
# return missing_data

# NAs(train).head()

There are lots of ways to deal with missing values.  
We can drop the columns with too many missing values or impute them instead.  

I don't really like to drop any columns since we lose data that way, so we'll impute missing values and maybe drop just one unnecessary column.

In [ ]:
def fill_and_transfrom(data):
    global cols_non_numeric
    global cols_numeric
    cols_numeric = []
    cols_non_numeric = []
    cpy = pd.DataFrame(data)
    cpy["PoolQC"] = cpy["PoolQC"].fillna("None")
    cpy["MiscFeature"] = cpy["MiscFeature"].fillna("None")
    cpy["Alley"] = cpy["Alley"].fillna("None")
    cpy["Fence"] = cpy["Fence"].fillna("None")
    cpy["FireplaceQu"] = cpy["FireplaceQu"].fillna("None")
    cpy["LotFrontage"] = cpy.groupby("Neighborhood")["LotFrontage"].transform(
        lambda x: x.fillna(x.median())
    )
    for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
        cpy[col] = cpy[col].fillna('None')
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        cpy[col] = cpy[col].fillna(0)
    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        cpy[col] = cpy[col].fillna(0)
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        cpy[col] = cpy[col].fillna('None')
    cpy["MasVnrType"] = cpy["MasVnrType"].fillna("None")
    cpy["MasVnrArea"] = cpy["MasVnrArea"].fillna(0)
    cpy['MSZoning'] = cpy['MSZoning'].fillna(cpy['MSZoning'].mode()[0])
    cpy = cpy.drop(['Utilities'], axis=1)
    cpy["Functional"] = cpy["Functional"].fillna("Typ")
    cpy['Electrical'] = cpy['Electrical'].fillna(cpy['Electrical'].mode()[0])
    cpy['KitchenQual'] = cpy['KitchenQual'].fillna(cpy['KitchenQual'].mode()[0])
    cpy['Exterior1st'] = cpy['Exterior1st'].fillna(cpy['Exterior1st'].mode()[0])
    cpy['Exterior2nd'] = cpy['Exterior2nd'].fillna(cpy['Exterior2nd'].mode()[0])
    cpy['SaleType'] = cpy['SaleType'].fillna(cpy['SaleType'].mode()[0])
    cpy['MSSubClass'] = cpy['MSSubClass'].fillna("None")

    # cols_numeric = []
    # cols_non_numeric = []
    for c in cpy.columns:
        if cpy[c].dtype == object:
            cols_non_numeric += [c]
            # lbl = LabelEncoder()
            # cpy[c] = lbl.fit_transform(cpy[c].values.tolist())
            cpy[c] = cpy[c].astype('category')
            # cpy[c] = cpy[c].astype('category').cat.codes.astype('category')
        else:
            if 'qual' in c.lower() or 'cond' in c.lower() or 'type' in c.lower() or 'class' in c.lower():
                cols_non_numeric += [c]
                cpy[c] = cpy[c].astype('category')
            else:
                cols_numeric += [c]
    # print('Shape data: {}'.format(cpy.shape))
    return cpy

# data_filled = fill_and_transfrom(data)

appendToPipeline(pipeline, fill_and_transfrom)
data_filled = pipeline.fit_transform(data)

In [ ]:
data_filled.head()

In [ ]:
data_filled.info()

## Feature Engineering

One feature that we can add is `TotalSqfootage`:

In [ ]:
def addTotalSq(data):
    cpy = pd.DataFrame(data)
    cpy['TotalSF'] = cpy['TotalBsmtSF'] + cpy['1stFlrSF'] + cpy['2ndFlrSF']
    return cpy

appendToPipeline(pipeline, addTotalSq)
data_filled = pipeline.fit_transform(data)

~~For an affective ML model we're going to need to standardize the features by removing the mean and scaling to unit variance.~~  

~~The standard score of a sample `x` is calculated as:~~  

~~z = \frac{(x - \mu)}{std}~~

For an affective ML model we're going to need to normalize the features by removing the mean and scaling to unit variance.  

The normalization of a sample `x` is calculated as follows:
$$
norm(x_i) = \frac{x_i − min(X)}{max(X) − min(X)}
$$

We would transfrom each column except for the target, since we need it's real values as predictions & validations.  

We can normalize the X vector while fitting the data to the model using:
```py
LinearRegression.fit(X, y, normalize=True)
```

Now we can split our `data` back into `train` and `test`

In [ ]:
# train = pd.read_csv('./data/train.csv', index_col='Id')
# test = pd.read_csv('./data/test.csv', index_col='Id')

appendToPipeline(pipeline, lambda data: pd.get_dummies(data), 'get_dummies')

# train_dummy = pd.get_dummies(data_filled)[:ntrain]
# test_dummy = pd.get_dummies(data_filled)[:ntest]
# test_dummy.index = test.index

# train_no_dummy = data_filled[:ntrain]
# test_no_dummy = data_filled[:ntest]

train_dummy = pipeline.fit_transform(train)
test_dummy = pipeline.fit_transform(test)

display(train_dummy.head())
display(train_dummy['SalePrice'].head())
display(train_dummy.dtypes)

# display(test_dummy.head())

## Training & Model Comparing

~~We're going to train our model on different sizes of train and validation sets.  
After that we will see which size of train set is the best.~~

As hyper-paramater testing we will choose different sized feature sets.  

That way we could control the polynomial degree of our model, which in turn will affect the model's flexibility.

In [ ]:
def gen_k(start, step, length):
    lst = []
    for i in range(length):
        lst += [start + (step*i)]
    return lst
    
    # def inner(start, step, size):
    #     curr = start
    #     while size != 0:
    #         yield curr
    #         curr += step
    #         size -= 1
    # return [i for i in inner(start, step, size)]

In [ ]:
# k = gen_k(start=16, step=6, length=12)
# k

In [ ]:
# SalePrice correlation matrix
train_dummy = pipeline.fit_transform(train)
corr = train_dummy.corr()
feats = []

# k = number of diffrent feature sets to choose (polynomial degree)
k = gen_k(start=16, step=6, length=12) # => [16, 22, ..., 82]

# picking the top correlated features, not including the target
for n in k:
    # cols = np.abs(corr).nlargest(n+1, 'SalePrice')['SalePrice'].index.tolist()
    cols = corr.nlargest(n+1, 'SalePrice')['SalePrice'].index.tolist()
    cols.remove('SalePrice')
    feats += [cols]

In [ ]:
# feats[0]

~~Let's try the sklearn backwards feature selection method, `RFE`.~~

The feature selection based on correletion to the target worked much better than `RFE`.

In [ ]:
# # est = SGDRegressor(learning_rate='optimal')
# # est = SGDRegressor()
# est = LinearRegression(normalize=True)

# data = pipeline.fit_transform(train)
# y = data['SalePrice']
# X = data.drop(columns=['SalePrice'])
# feats = []

# # k = number of diffrent feature sets to choose (polynomial degree)
# k = gen_k(start=10, step=7, length=4) # => [10, 17, 24, 31, 38, 45, 52, 59, 66, 73]

# for n in tqdm(k):
#     rfe = RFE(
#         estimator = est,
#         n_features_to_select = n
#     )
#     rfe.fit_transform(X, y)
#     # feats += [ X.columns[rfe.support_].tolist() ]
#     feats += [ X.columns[rfe.support_ == False].tolist() ]

# print('done')

In [ ]:
# feats[0]

In [ ]:
# plot one of the different heatmaps we chose in the last cell
plt.figure(figsize=(24,11))
sns.set(font_scale=1.25)

cols = ['SalePrice'] + feats[1]
cm = np.corrcoef(train_dummy[cols].values.T)
hm = sns.heatmap(
    cm,
    cbar = True,
    annot = True,
    square = True,
    fmt = '.2f',
    cmap = 'coolwarm',
    yticklabels = cols,
    xticklabels = cols,
    # title = f'Top {len(feats)-1} Correlated Features',
    annot_kws = {'size': 11}
)

hm.set(title=f'Top {len(cols)-1} Correlated Features')
plt.show()

In [ ]:
def lrmse(y_true, y_pred, squared):
    return metrics.mean_squared_error(np.log(y_true), np.log(y_pred), squared=squared)

lrmse_socrer = make_scorer(lrmse, greater_is_better=False, squared=True)
lmse_socrer = make_scorer(lrmse, greater_is_better=False, squared=False)

In [ ]:
### ESTIMATORS ###
estimators = []
# estimators += [SGDRegressor(learning_rate='optimal')]
# estimators += [SGDRegressor()]
# estimators += [LinearRegression(normalize=True)]
# estimators += [{'model':Ridge, 'kws':{'normalize':True, 'alpha':0.5}}]
estimators += [{'model':Ridge, 'kws':{'normalize':True, 'alpha':0.29}}]
estimators += [{'model':Ridge, 'kws':{'normalize':True, 'alpha':0.2}}]
# estimators += [{'model':Ridge, 'kws':{'normalize':True, 'alpha':0.05}}]
# estimators += [ElasticNet(normalize=True, alpha=0.5)]
# estimators += [Lasso(normalize=True, alpha=0.5)]
# estimators += [Lasso(normalize=True, alpha=0.2)]
### ESTIMATORS ###

### CV METHODS ###
# l1o = LeavePOut(p=1)
# fivefold = KFold(n_splits=5, shuffle=True, random_state=101)
tenfold = KFold(n_splits=10, shuffle=True, random_state=101)
### CV METHODS ###

scores = {}
# scores = dict()

for d in estimators:
    model = d['model']
    kws = d['kws']
    est = model(**kws)
    d['est'] = est

    est_name = ""
    for i in str(est.__repr__).strip("<>'").split(' ')[-2:]:
        est_name += i.strip('of ')
    scores[est_name] = []

    for cols in tqdm(feats):
        data = pipeline.fit_transform(train)
        X = data[cols]
        y = data['SalePrice']
        res = cross_validate(
            X = X,
            y = y,
            estimator = est,
            cv = tenfold.split(X),
            # cv = fivefold.split(X),
            # cv = l1o.split(X),
            return_train_score = True,
            return_estimator = True,
            scoring = {
                'neg_LRMSE': lrmse_socrer,
                'neg_LMSE': lmse_socrer,
                'neg_MSE': 'neg_mean_squared_error',
                # 'neg_MAE': 'neg_mean_absolute_error',
                # 'neg_MSLE': 'neg_mean_squared_log_error'
            }
        )
        res['feats'] = cols
        scores[est_name] += [res]

print('\n\ndone')

In [ ]:
est_names = list(scores.keys())
print('estimators:\n', est_names)
print('\nscores:\n', list(scores[est_names[0]][0].keys()))

In [ ]:
# defs for easier access to loss function
# MSE = 'neg_mean_squared_error'
# LRMSE = 'neg_log_root_mean_squared_error'
# MAE = 'neg_mean_absolute_error'
# MSLE = 'neg_mean_squared_log_error'

MSE = 'MSE'
LRMSE = 'LRMSE'
LMSE = 'LMSE'

# losses = [MSE, RMSE, MAE, MSLE]
# losses = [MSE, LRMSE, MAE, MSLE]
losses = [MSE, LRMSE, LMSE]

Now we're going to need to choose the best estimator among the estimators we tried.  

Let's go over them in a clean df, each score in this df is the mean of all scores that estimator got.

In [ ]:
cols = [f'test_{loss}' for loss in losses]
estimators_df = pd.DataFrame(columns=cols, index=est_names)

for est in est_names:
    s = scores[est]
    scores_df = pd.DataFrame(s)

    for loss in losses:
        estimators_df[f'test_{loss}'][est] = np.mean(-scores_df[f'test_neg_{loss}'].apply(np.mean))

estimators_df

In [ ]:
# we'll choose the best estimator

# loss = [MSE, 'MSE']
# loss = [LRMSE, 'LRMSE']
loss = [LMSE, 'LMSE']

best_score = np.min(estimators_df[f'test_{loss[0]}'])
best_est = estimators_df[ estimators_df[f'test_{loss[0]}']==best_score ].index[0]

print('the best estimator is:\n', best_est)

Let's choose one of the models and take a look of what we got so far.  

We could plot the difference between the predictions and the true value of the target.  
In a perfect world we would get a 45deg function ($f_{(x)}=x$ ➜ $y_{pred}=y_{true}$)

In [ ]:
plt.figure(figsize=(15,10))

def plot_predictions(data, feats, model, title=None, axis=None):
    if title == None:
        title = f'deg={len(feats)}'

    df = pd.DataFrame({
        'y_true': data['SalePrice'],
        'y_pred': model.predict(data[feats])
    })
    g = sns.regplot(
        x = 'y_true',
        y = 'y_pred',
        data = df,
        line_kws = {'color': '#B55D60'},
        scatter_kws = {'edgecolor': 'white'},
        ax = axis
    )
    g.set(title = title)
    return g

s = random.choice(scores[best_est])
cols = s['feats']
model = random.choice(s['estimator'])
train_dummy = pipeline.fit_transform(train)

plot_predictions(train_dummy, cols, model, title=f'Estimator = {best_est}\nPoly_Degree = {len(cols)}')
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=math.ceil(len(k)/3), ncols=3, sharex=True, sharey=True) ##sharex=True, sharey=True##
fig.set_size_inches(17, 22)
# plt.figure(figsize=(16,16))
fig.text(0.5, 0.04, 'y_true', ha='center')
fig.text(0.04, 0.5, 'y_pred', va='center', rotation='vertical')
axs = axs.flatten().tolist()

train_dummy = pipeline.fit_transform(train)

for s in tqdm(scores[best_est]):
    cols = s['feats']
    model = random.choice(s['estimator'])

    g = plot_predictions(train_dummy, cols, model, title=f'Estimator = {best_est}\nPoly_Degree = {len(cols)}', axis=axs.pop(0))
    g.set(xlabel='', ylabel='')

fig.tight_layout()
plt.show()

In [ ]:
scores_df = pd.DataFrame(scores[best_est]).drop(columns=['fit_time', 'score_time', 'estimator'])

for loss in losses:
    scores_df[f'test_{loss}'] = -scores_df[f'test_neg_{loss}'].apply(np.mean)
    scores_df[f'train_{loss}'] = -scores_df[f'train_neg_{loss}'].apply(np.mean)
    scores_df = scores_df.drop(columns=[f'test_neg_{loss}', f'train_neg_{loss}'])

# scores_df[f'test_{RMSE}'] = np.sqrt(scores_df[f'test_{MSE}'])
# scores_df[f'train_{RMSE}'] = np.sqrt(scores_df[f'train_{MSE}'])

scores_df['deg'] = scores_df['feats'].apply(len)

scores_df

In [ ]:
def plot_loss(loss, x, y, data=scores_df):
    sns.lineplot(
        x = 'deg',
        y = f'{y}_{loss[0]}',
        data = scores_df,
        # data = np.log(scores_df[['deg', f'{y}_{loss[0]}']]),
        marker = 'o'
    )

    # plt.title('Err Plot')
    plt.xlabel('Degree of Polynomial')
    plt.ylabel(f'Loss = {loss[1]}')

In [ ]:
plt.figure(figsize=(15,8))
# loss = [MSE, 'MSE']
# loss = [LRMSE, 'LRMSE']
loss = [LMSE, 'LMSE']
# loss = [MAE, 'MAE']
# loss = [MSLE, 'MSLE']

plot_loss(
    loss,
    x = 'deg',
    y = 'train',
    data = scores_df
)

plot_loss(
    loss,
    x = 'deg',
    y = 'test',
    data = scores_df
)

plt.title(f'Estimator = {best_est}')
plt.legend(['train', 'test'], fontsize='large')
plt.show()

## Testing

By looking at the graph above we can notice that the best model we fitted had around 80 features.  

Let's choose that model and try and predict the actual test data from kaggle.

In [ ]:
# loss = MSE
loss = LMSE

# get the best score's feature list
best_score = min(scores_df[f'test_{loss}'])
i = scores_df[scores_df[f'test_{loss}'] == best_score].index.tolist()[0]
cols = scores[best_est][i]['feats']

# get a clean train & test DFs
train_dummy = pipeline.fit_transform(train)
test_dummy = pipeline.fit_transform(test)

# train the model on the entire train set
# for i,est in enumerate(estimators):
for d in estimators:
    est = d['est']
    if best_est in str(est.__repr__):
        break
# est = estimators[i]
model = est.fit(train_dummy[cols], train_dummy['SalePrice'])

# predict the test set
y_pred = model.predict(test_dummy[cols])

pred = pd.DataFrame(
    y_pred,
    columns = ['SalePrice'],
    index = test.index
)

display(pred)

In [ ]:
# Get the fitted parameters used by the function
plt.figure(figsize=(16,8))

(mu, sigma) = ( round(item, 2) for item in stats.norm.fit(pred) )

display(Markdown(f'$\mu$ = {mu}'))
display(Markdown(f'$\sigma$ = {sigma}'))

# plot the distribution
sns.distplot(
    pred,
    kde_kws = {'color': '#4C4C4C'}
)
plt.legend(
    [f'y_pred ~ $N(\mu=${mu}, $\sigma=${sigma}$)$'],
    fontsize = 'x-large'
)
# plt.ylabel('Frequency')
plt.title('Prediction Distribution')

In [ ]:
# # loss = MSE
# loss = LMSE

# # get the best score's feature list
# best_score = min(scores_df[f'test_{loss}'])
# i = scores_df[scores_df[f'test_{loss}'] == best_score].index.tolist()[0]
# cols = scores[best_est][i]['feats']

# # get a clean train & test DFs
# # train_dummy = pipeline.fit_transform(train)
# test_dummy = pipeline.fit_transform(test)

# # train the model on the entire train set
# # for i,est in enumerate(estimators):
# # for est in estimators:
# #     if best_est in str(est.__repr__):
# #         break
# # est = estimators[i]
# # model = est.fit(train_dummy[cols], train_dummy['SalePrice'])

# # scores[best_est][i]['estimator']

# # predict the test set
# preds = pd.DataFrame(columns=[i for i in range(len(scores[best_est][i]['estimator']))])
# for i,model in  enumerate(scores[best_est][i]['estimator']):
#     preds[i] = model.predict(test_dummy[cols])
# preds

# y_pred = []
# for i in preds.index:
#     y_pred += [np.mean(preds.iloc[i])]

# pred = pd.DataFrame(
#     y_pred,
#     columns = ['SalePrice'],
#     index = test.index
# )

# display(pred)

In [ ]:
pred.to_csv('pred.csv')

## Summary

### Screenshots

![submissions](./screenshots/submissions.png)  

![leaderboards](./screenshots/leaderboards.png)

### Conclusions

## References

1. 